This notebook aims at creating the dataset from the list of archives urls obtained via get_archives.sh.<br>
Running time can take several hours.

In [1]:
#enter path to text file containing archives list
path_19_01_15_17 = ''

In [2]:
#For a first expeeriment the wat files were retrieved first. This is to be modified
urls_wat = path_19_01_15_17.read().splitlines()

In [4]:
urls_wet = list(map(lambda x: x.replace('wat', 'wet'), urls_wat))

In [5]:
urls_wet[0:5]

['https://data.commoncrawl.org/crawl-data/CC-MAIN-2019-04/segments/1547583656530.8/wet/CC-MAIN-20190115225438-20190116011438-00000.warc.wet.gz',
 'https://data.commoncrawl.org/crawl-data/CC-MAIN-2019-04/segments/1547583656530.8/wet/CC-MAIN-20190115225438-20190116011438-00001.warc.wet.gz',
 'https://data.commoncrawl.org/crawl-data/CC-MAIN-2019-04/segments/1547583656530.8/wet/CC-MAIN-20190115225438-20190116011438-00002.warc.wet.gz',
 'https://data.commoncrawl.org/crawl-data/CC-MAIN-2019-04/segments/1547583656530.8/wet/CC-MAIN-20190115225438-20190116011438-00003.warc.wet.gz',
 'https://data.commoncrawl.org/crawl-data/CC-MAIN-2019-04/segments/1547583656530.8/wet/CC-MAIN-20190115225438-20190116011438-00004.warc.wet.gz']

In [7]:
from warcio.archiveiterator import ArchiveIterator
import json
import pandas as pd
import csv
import requests

In [8]:
archive_links = []
target_uris = []
warc_record_ids = []
content_lengths = []
warc_dates = []
texts = []

In [9]:
def get_archive_content(path,idx):

  wet_path = path

  try:
    resp = requests.get(wet_path, stream=True)

    records = ArchiveIterator(resp.raw, arc2warc=True)
  except:
    print(f'problem to retrieve archive content {wet_path}')

  for record in records:
    
        try:
            url = record.rec_headers.get_header('WARC-Target-URI')
        
            if 'blackrock' in url.lower():

                archive_links.append(wet_path)
                
                target_uris.append(url)

                try:
                  content_length = record.rec_headers.get_header('Content-Length')
                  content_lengths.append(content_length)

                except:
                  content_lengths.append(float("nan"))

                try:
                  warc_date = record.rec_headers.get_header('WARC-Date')
                  warc_dates.append(warc_date)

                except:
                  warc_dates.append(float("nan"))

                try:
                  content = record.content_stream().read()
                  texts.append(content.decode('utf-8'))
                
                except:
                  texts.append(float("nan"))

                

                try:
                  warc_record = record.rec_headers.get_header('WARC-Refers-To')
                  warc_record_ids.append(warc_record)
                
                except:
                  warc_record_ids.append(float("nan"))



        except:
            pass        
                    

In [ ]:
for idx, path in enumerate(urls_wet):
  get_archive_content(path,idx)

In [ ]:
print(len(archive_links))
print(len(target_uris))
print(len(warc_record_ids))
print(len(content_lengths))
print(len(warc_dates))
print(len(texts))

In [ ]:
df_urls = pd.DataFrame({'archive_links':archive_links,'urls':target_uris,'record_ids':warc_record_ids,'content_lengths':content_lengths,'warc_dates':warc_dates, 'texts':texts})

In [ ]:
df_urls.head()

In [ ]:
#df_urls.to_csv(' ',index=False)